In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
# get pandas
import pandas as pd
# we'll do some visual checks, get the necessary packages
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
# done, save the model
import pickle

In [2]:
# first load the model
model = pickle.load(open('../models/gbt_reg_v2.pkl', 'rb'))

In [3]:
# now load the test data
df_raw = pd.read_csv('../data/test.csv')

In [4]:
# wrapper function to perform some simple cleanup
def clean_df(df):
    # keep only certain variable types
    df = df.select_dtypes(include=['int64','float64'])
    # drop everything with more than 10% of missing values
    df = df[df.columns[df.isnull().mean() < 0.1]]
    # looks like Garage built goes with Year built, replace NA using that
    df.loc[df['GarageYrBlt'].isnull(),'GarageYrBlt'] = df['YearBuilt']
    # put zero for MasVnrArea
    df.loc[df['MasVnrArea'].isnull(),'MasVnrArea'] = 0.0
    return df

In [5]:
df = clean_df(df_raw)

In [6]:
# check quickly n/a
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# show columns with most missing entries
missing_data.head(10)

,Total,Percent
BsmtHalfBath,2,0.001371
BsmtFullBath,2,0.001371
TotalBsmtSF,1,0.000685
BsmtUnfSF,1,0.000685
BsmtFinSF2,1,0.000685
BsmtFinSF1,1,0.000685
GarageCars,1,0.000685
GarageArea,1,0.000685
MasVnrArea,0,0.000000
YearRemodAdd,0,0.000000


In [7]:
# fill na in special case with rule
# use 1stFlrSF for bsmt when null
df.loc[df['TotalBsmtSF'].isnull(),'TotalBsmtSF'] = df['1stFlrSF']
# similar for garage but add factor 1/2
df.loc[df['GarageArea'].isnull(),'GarageArea'] = df['1stFlrSF']*0.5

In [8]:
# take care of the rest with 0s
df = df.fillna(0)

In [9]:
# recheck
# check quickly n/a
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# show columns with most missing entries
missing_data.head(10)

,Total,Percent
YrSold,0,0.0
MoSold,0,0.0
GrLivArea,0,0.0
LowQualFinSF,0,0.0
2ndFlrSF,0,0.0
1stFlrSF,0,0.0
TotalBsmtSF,0,0.0
BsmtUnfSF,0,0.0
BsmtFinSF2,0,0.0
BsmtFinSF1,0,0.0


In [10]:
# prep scikit
input_features = list(df.columns)
input_features.remove('Id')
X = df[input_features].values

In [11]:
y = model.predict(X)

In [12]:
# store output
df['log_SalePrice'] = y

In [13]:
# get the original variable 
df['SalePrice'] = np.exp(df['log_SalePrice']) - 1

In [14]:
# drop the log var
df = df.drop(['log_SalePrice'], axis=1)

In [15]:
df.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1461,20,11622,5,6,1961,1961,0.0,468.0,144.0,...,140,0,0,0,120,0,0,6,2010,125772.429183
1,1462,20,14267,6,6,1958,1958,108.0,923.0,0.0,...,393,36,0,0,0,0,12500,6,2010,153754.595432
2,1463,60,13830,5,5,1997,1998,0.0,791.0,0.0,...,212,34,0,0,0,0,0,3,2010,185200.795722
3,1464,60,9978,6,6,1998,1998,20.0,602.0,0.0,...,360,36,0,0,0,0,0,6,2010,185872.619142
4,1465,120,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,82,0,0,144,0,0,1,2010,186829.662470


In [16]:
# save the file for submission
df[['Id','SalePrice']].to_csv('../data/test_submission_180225.csv',index=False)